In [15]:
import numpy as np

# Introducción

Random sample consensus (RANSAC) es un método iterativo para calcular los parámetros de un modelo matemático de un conjunto de datos observados que contiene valores atípicos. Es un algoritmo no determinista en el sentido de que produce un resultado razonable sólo con una cierta probabilidad, mayor a medida que se permiten más iteraciones [Más información en wikipedia](https://es.wikipedia.org/wiki/RANSAC)

Lo que hace ransac es básicamente elegir aleatoriamente un número n de muestras, crear el modelo que se ajuste a dichas n muestras y probar el modelo con todas las demás muestras. Si el error con las demas muestras es bajo podemos determinar que es un modelo válido, sino volvemos a repetir el proceso hasta que encontremos un modelo que se ajuste.

En este caso vamos a aplicar RANSAC para el cálculo de una matriz de transformación (o matriz homográfica). Como entrada tenemos una serie de puntos de origen y de destino. Queremos encontrar una matriz que sirva para realizar transformaciones de los puntos de origen a los puntos de salida. Adicionalmente en esta serie de puntos de entrada y salida podemos tener valores atípicos o outliers que para nada deben tenerse en cuenta para el calculo anteriormente mencionado.

Por cada match que encontremos tenemos la siguiente ecuación que nos relaciona los puntos de origen (x,y) en los puntos de destino (u,v)

\begin{align}
\begin{pmatrix} 
u\\
v
\end{pmatrix}
=
\begin{pmatrix} 
m_1 & m_2 \\
m_3 & m_4
\end{pmatrix}
\begin{pmatrix} 
x \\
y
\end{pmatrix}
+
\begin{pmatrix} 
t_x\\
t_y
\end{pmatrix} 
\end{align}

Para una transformación afín necesitamos 3 o match. Cada match contribuye con 2 ecuaciones al siguiente sistema de ecuaciones de la forma:

$$Ax=b$$

\begin{align}
\begin{pmatrix} 
  x_1     & y_1 & 0 & 0 & 1 & 0\\ 
  0 & 0 & x_1 & y_2 & 0 & 1  \\
  x_2     & y_2 & 0 & 0 & 1 & 0\\ 
  0 & 0 & x_2 & y_2 & 0 & 1 \\
  x_3     & y_3 & 0 & 0 & 1 & 0\\ 
  0 & 0 & x_3 & y_3 & 0 & 1 
\end{pmatrix}
*
\begin{pmatrix} 
  m_1\\ 
  m_2\\
  m_3\\
  m_4\\
  t_2\\ 
  t_2 
\end{pmatrix}
=
\begin{pmatrix} 
  u_1\\ 
  v_1\\
  u_2\\
  v_2\\
  u_3\\ 
  v_4 
\end{pmatrix}
\end{align}

Esta ecuación la podemos resolver por mínimos cuadrados de la siguiente forma

$$ x = [A^{T}A]^{-1}A^{T}b $$

Por tanto y a modo de resumen:

* Elegimos 3 puntos aleatorios
* Resolvemos el sistema $Ax=b$
* Obtenemos todos los puntos que concuerden con el modelo (dando un margen de error)
* Con todos los inliers encontrados volvemos a resolver la ecuación $Ax=b$ para refinar el modelo
* Seguimos iterando un número iterations de veces por si encontramos un modelo que contenga mas inliers que el encontrado previamente

In [22]:
def ransac(src, dst, error, iterations=1):
    i = 0
    best_model = None
    best_affine_transform = None
    max_inliers = 0
    inliers_founds = np.array([])
    while i < iterations:
        # Elegimos aleatoriamente 3 muestras
        choices = np.random.choice(len(dst), 3, replace=False)
        selected_dst = dst[choices]
        selected_src = src[choices]
        selected_src = selected_src.reshape((3,2))
        selected_dst = selected_dst.reshape((3,2))

        # Formamos el sistema de ecuaciones Ax=b
        A = np.array([[selected_src[0][0], selected_src[0][1],0,0,0,1],
                    [0,0,selected_src[0][0], selected_src[0][1], 1,0],
                    [selected_src[1][0], selected_src[1][1],0,0,0,1],
                    [0,0,selected_src[1][0], selected_src[1][1], 1,0],
                    [selected_src[2][0], selected_src[2][1],0,0,0,1],
                    [0,0,selected_src[2][0], selected_src[2][1], 1,0]])
        b = np.array([selected_dst[0][0], selected_dst[0][1], selected_dst[1][0], selected_dst[1][1], selected_dst[2][0], selected_dst[2][1]])
        # Resolvemos el sistema de ecuaciones. X=(A^TA)^-1 * b
        X = np.linalg.inv(A.T @ A) @ A.T @ b
        affine_transform = np.array([[X[0], X[1], X[4]],
                                    [X[2], X[3], X[5]],
                                    [0, 0, 1]])
        # Pasamos por todos los puntos y calculamos el numero de inliers y los inliers como tal
        number_inliers = 0
        inliers = []
        for point_src, point_dst in zip(src, dst):
            predicted_dst = affine_transform @ np.hstack((point_src,[[1]])).T.reshape(3,1)
            dist = np.linalg.norm(point_dst-(predicted_dst[:2]).reshape((1,2)))
            if dist < error:
                inliers.append(np.concatenate((point_src, point_dst)))
                number_inliers += 1
        # Si el número de inliers es mejor que el mejor obtenido en el pasado consideramos este modelo como mejor
        if number_inliers > max_inliers:
            inliers_founds = np.array(inliers)
            best_model = affine_transform
            best_affine_transform = affine_transform
            max_inliers = number_inliers
        i += 1
    # Ajusta de nuevo el modelo con todos los inliers detectados
    A = []
    b = []
    for inlier in inliers_founds:
        A.append([inlier[0][0], inlier[0][1], 0, 0, 0, 1])
        A.append([0, 0, inlier[0][0], inlier[0][1], 1, 0])
        b.append(inlier[1][0])
        b.append(inlier[1][1])
        pass
    A = np.array(A)
    b = np.array(b)
    X = np.linalg.inv(A.T @ A) @ A.T @ b
    #Devolvemos la transformación afín
    affine_transform = np.array([[X[0], X[1], X[4]],
                                [X[2], X[3], X[5]],
                                [0, 0, 1]])
    return affine_transform

A continuación creamos una lista de puntos que contienen tanto inliers como outliers y pasamos los puntos de origen y destino al método de calculo de la matriz afín mediante ransac. Si nos fijamos los puntos origen (tanto en la coordenada x como y) han sido formando sumando 100 a los puntos de origen, exceptuando aquellos que sean outliers.

Podemos observar que la matriz que encuentra es (realmente aparecen número muy pequeños que se pueden aproximar a 0)

\begin{pmatrix}
1 & 0 & 100\\
0 & 1 & 100\\
0 & 0 & 1
\end{pmatrix}

posteriormente se imprime el primer punto de origen
\begin{pmatrix}
0\\
0\\
1
\end{pmatrix}

y el resultado de multiplicar dicho punto por la matriz de transformación

\begin{align}
\begin{pmatrix}
1 & 0 & 100\\
0 & 1 & 100\\
0 & 0 & 1
\end{pmatrix}
*
\begin{pmatrix}
0\\
0\\
1
\end{pmatrix}
=
\begin{pmatrix}
100\\
100\\
1
\end{pmatrix}
\end{align}

El hecho de añadir un 1 al final de los puntos es para convertirlos en coordenadas homogeneas y poder realizar la operación de rotación y translación mediante una única matriz.

In [23]:
src_pts = np.float32([ [0,0],[100,2],[90,80],[20,100], [3,4],[5.6, 7] ]).reshape(-1,1,2)
dst_pts = np.float32([ [100,100],[200,102],[190,180],[120,200], [453,453], [22.2,453] ]).reshape(-1,1,2)
# Calculamos la transformación afín mediante RANSAC
M = ransac(src_pts,dst_pts,10,100)
print("matrix")
print(M)
print(np.hstack((src_pts[0],[[1]])).T.reshape(3,1))
print(M @ np.hstack((src_pts[0],[[1]])).T.reshape(3,1))

matrix
[[  1.00000000e+00   3.88578059e-16   1.00000000e+02]
 [ -2.22044605e-16   1.00000000e+00   1.00000000e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[[ 0.]
 [ 0.]
 [ 1.]]
[[ 100.]
 [ 100.]
 [   1.]]
